# Agentic AI with AutoGen

Agentic AI refers to artificial intelligent systems that form an "agency". This enables them to perform specific tasks and make decisions to achieve specific goals. These AI systems can adapt to changes and execute decisions without continuos human interventions. 

AutoGen is an open-source framework that facilitates the creation and orchestration of Agentic AI systems. Autogen makes it possible to develop flexible multi agent systems that can interact with each other. These agents can operate both autonomousoly and with human oversight. 

AutoGen can: 
- Facilitate mathematical problem solving and code generation.
- Enhance the quality and accuracy of information retrieval.
- Optimize complex decision making processes.



## Lab Description:

This lab explores agentic AI using AutoGen, showcasing how AI agents can autonomously interact, collaborate, and execute tasks. The lab begins by introducing basic conversable agents and making them communicate with each other. Next, we incorporate a human-in-the-loop to allow human participation in the conversation. We then demonstrate a Code Executor Agent, capable of running code autonomously. Finally, we set up a group chat involving multiple specialized agents, including a Coder Agent, Critique Agent, and User Proxy Agent, to showcase how agents can collaborate effectively on complex tasks.

## Lab Objectives

- Understand the fundamentals of AutoGen and conversable AI agents.
  
- Enable AI agents to communicate and collaborate autonomously.
  
- Integrate a human-in-the-loop to participate in AI-driven conversations.
  
- Demonstrate a Code Executor Agent and a multi-agent group chat with specialized roles.

## Agent

In AutoGen, an agent is the entity that can send and recieve messages from other agents. An agent can be powered by humans, models or code executors. An example of a built in agent in AutoGen is the `ConversableAgent`.

Let us build a simple 2 agent system that discusses about 'Large Language Models' with each other.



First step would be to configure the LLM. We use llama3.1:8b from ollama. We will have to edit the `config_list`.

In [13]:
config_list = [
    {
        # Let's choose the Meta's Llama 3.1 model (model names must match Ollama exactly)
        "model": "llama3.1:8b",
        # We specify the API Type as 'ollama' so it uses the Ollama client class
        "api_type": "ollama",
        "stream": False,
        #Specify the address where ollama is hosted
        "client_host": "http://10.79.253.112:11434",
    }
]

Once we have our LLM configured, we are all set to initialize the agents.

In [14]:
from autogen import ConversableAgent

luna = ConversableAgent(
    "luna",
    system_message="Your name is Luna and you are a part of a duo that is discussing about Large Language Models. Your conversation should be brief and concise.",
    llm_config={"config_list": config_list},
    human_input_mode="NEVER",  # Never ask for human input.
)

zara = ConversableAgent(
    "zara",
    system_message="Your name is Zara and you are a part of a duo that is discussing about Large Language Models. Your conversation should be brief and concise.",
    llm_config={"config_list": config_list},
    human_input_mode="NEVER",  # Never ask for human input.
)


In [ ]:
We can initiate the conversation now that we have initialized the agents.

In [15]:
result = luna.initiate_chat(zara, message="Zara, tell me about the latest innovations in Large Language Models.", max_turns=2)

luna (to zara):

Zara, tell me about the latest innovations in Large Language Models.

--------------------------------------------------------------------------------
zara (to luna):

I'm thrilled to share with you what Alex and I have been working on! We've been following the latest advancements in Large Language Models, and there are some exciting developments.

Recently, researchers have made significant progress in scaling up models like transformer-XL and Longformer. These models have improved performance on a wide range of tasks, including language translation, text summarization, and conversational AI.

One notable innovation is the use of sparse attention mechanisms, which allow for more efficient computation while maintaining accuracy. This has enabled researchers to train larger models with billions of parameters, pushing the state-of-the-art in natural language processing.

We've also been experimenting with multimodal Large Language Models that can understand and generate 

We have provided system instructions to both the agents that they are each part of a duo conversing about Large Language Models. We have powered them with the `llm_config`. During initiation we set `max_turns` to 2, which terminates the chat after two turns.

### The flow is summarized by the following diagram:
<br><br>
<img src="./dia#1.jpg" alt="Alt Text" width="650" height="650" style="display: block; margin: 0 auto;"/>

## Human in the loop 

So what if you want to converse with an agent ?, say you want to talk with Zara about LLMs. You can initialize an agent powered by human. You can do this by setting `human_input_mode="ALWAYS"` in the agent's initialization. This will prompt you for an input at every turn of the conversation. Let's see this in action. 

In [16]:
zara = ConversableAgent(
    "zara",
    system_message="Your name is Zara and you are a part of a duo that is discussing about Large Language Models. Your conversation should be brief and concise.",
    llm_config={"config_list": config_list},
    human_input_mode="NEVER",  # Never ask for human input.
)

human = ConversableAgent(
     "human",
     llm_config=False,    #no need to use LLM as the user provides the input
     human_input_mode="ALWAYS",  #prompts you for an input at every turn
)

We are all set to initiate the conversation !

In [18]:
result = human.initiate_chat(
    zara,  # specify that the chat is with Zara
    message="Hey, Zara, have you heard about the transformers model ? ",
)

human (to zara):

Hey, Zara, have you heard about the transformers model ? 

--------------------------------------------------------------------------------
zara (to human):

Yeah, I've been reading about it with my colleague. The transformer architecture has revolutionized the way we approach NLP tasks. It's amazing how a simple yet elegant design can outperform traditional recurrent neural networks in many cases. We're actually experimenting with it for our language understanding project. What are your thoughts on its limitations?

--------------------------------------------------------------------------------


Replying as human. Provide feedback to zara. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  i am not aware of any limitations as of now, it is really great, the attention mechanism blew my mind


human (to zara):

i am not aware of any limitations as of now, it is really great, the attention mechanism blew my mind

--------------------------------------------------------------------------------
zara (to human):

I know what you mean! The self-attention mechanism in transformers is a game-changer. It allows the model to weigh the importance of different input elements relative to each other, which is super powerful for capturing contextual relationships. We're seeing some impressive results with it too! Have you explored its applications in dialogue systems or text summarization?

--------------------------------------------------------------------------------


Replying as human. Provide feedback to zara. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  exit


### The flow can be summarized by the following diagram: 
<br>
<img src="./dia#2u.png" alt="Alt Text" width="650" height="650" style="display: block; margin: 0 auto;"/>
<br><br>


Human can either decide to reply, skip or exit. If human chooses to reply, the agent will continue the conversation accordingly. If the human decide to skip (press "Enter"), the agent will initiate auto reply. If human decides to exit, the conversation is terminated. 

## Code Executors

In AutoGen, a code executor is a component that takes in an input message which contain a code block, perform the execution and ouputs the result. Let us try to initialize a built-in command line code executor which executes code in the command line environment. 

When the executor recieves a code block, it first writes the code into a code file. Then it starts a subprocess to execute that code file. When it gets the console output, it finally reads the console output to give the final output message. 



First, we would want a temporary directory to store the code file.

In [1]:
import tempfile

# Create temporary directory to store the code file
temp_dir = tempfile.TemporaryDirectory()

Now we will initialize both the code executor agent and the local command line code executor. We will also pass the local command line code executor to the agent initialization. 

In [2]:
from autogen import ConversableAgent
from autogen.coding import LocalCommandLineCodeExecutor

executor = LocalCommandLineCodeExecutor(
    timeout=10,  # Timeout for each code execution in seconds.
    work_dir=temp_dir.name,  # Use the temporary directory to store the code files.
)

# Create an agent with code executor configuration.
code_executor_agent = ConversableAgent(
    "code_executor_agent",
    llm_config=False,  # Turn off LLM for this agent.
    code_execution_config={"executor": executor},  # Use the local command line code executor.
    human_input_mode="ALWAYS",  # Always take human input for this agent for safety.
)

Now we can pass a message with codeblock to the code executor agent. 

In [3]:
message = """This is a message with a code block.
Code block:
```python
def fibonacci(n):
    fib_sequence = [0, 1]
    for i in range(2, n):
        next_number = fib_sequence[i-1] + fib_sequence[i-2]
        fib_sequence.append(next_number)
    return fib_sequence[:n]

# Print the first 5 Fibonacci numbers
first_5_fibonacci = fibonacci(5)
print("First 5 Fibonacci numbers:", first_5_fibonacci)
```
End of Message"""

            


Now we will make the executor generate a response. 

In [4]:
reply = code_executor_agent.generate_reply(messages=[{"role": "user", "content": message}])
print(reply)

Replying as code_executor_agent. Provide feedback to the sender. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  



>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING CODE BLOCK (inferred language is python)...
exitcode: 0 (execution succeeded)
Code output: First 5 Fibonacci numbers: [0, 1, 1, 2, 3]



The model generated the output after executing the code using `LocalCommandLineCodeExecutor` from AutoGen.

## Group Chat

Group chats in AutoGen are multi-agent conversations. The `user -> coder -> critic` Group chat would be an excellent example. What happens in this Group chat is really simple. There is a user proxy agent who initiate a group chat and later on execute the code. If there is any error in the code, the user proxy agent will not be able to provide an output. First, the user proxy initiates a chat by giving a task. Then the coder agent provides code for the particular task. The code is then passed to the critic, who evaluates the code by searching for any logical, syntactical or conceptual erros. Then the user executes the code, if any errors are present, the error is outputted as the user proxy messsage. The coder agent analyses the response further increasing the quality of the code. You can set `max_turns` to control the number of times you want the turns to repeat. 

In [20]:
import autogen 

llm_config = {"config_list": config_list}

user = autogen.UserProxyAgent(
    name="User_proxy",
    system_message="A human admin.",
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": "groupchat",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
    human_input_mode="NEVER",
)
coder = autogen.AssistantAgent(
    name="Coder",  # the default assistant agent is capable of solving problems with code
    llm_config=llm_config,
)
critic = autogen.AssistantAgent(
    name="Critic",
    system_message="""Critic. You are a helpful assistant highly skilled in evaluating the quality of a given visualization code by providing a score from 1 (bad) - 10 (good) while providing clear rationale. YOU MUST CONSIDER VISUALIZATION BEST PRACTICES for each evaluation. Specifically, you can carefully evaluate the code across the following dimensions
- bugs (bugs):  are there bugs, logic errors, syntax error or typos? Are there any reasons why the code may fail to compile? How should it be fixed? If ANY bug exists, the bug score MUST be less than 5.
- Data transformation (transformation): Is the data transformed appropriately for the visualization type? E.g., is the dataset appropriated filtered, aggregated, or grouped  if needed? If a date field is used, is the date field first converted to a date object etc?
- Goal compliance (compliance): how well the code meets the specified visualization goals?
- Visualization type (type): CONSIDERING BEST PRACTICES, is the visualization type appropriate for the data and intent? Is there a visualization type that would be more effective in conveying insights? If a different visualization type is more appropriate, the score MUST BE LESS THAN 5.
- Data encoding (encoding): Is the data encoded appropriately for the visualization type?
- aesthetics (aesthetics): Are the aesthetics of the visualization appropriate for the visualization type and the data?

YOU MUST PROVIDE A SCORE for each of the above dimensions.
{bugs: 0, transformation: 0, compliance: 0, type: 0, encoding: 0, aesthetics: 0}
Do not suggest code.
Finally, based on the critique above, suggest a concrete list of actions that the coder should take to improve the code.
""",
    llm_config=llm_config,
)

groupchat = autogen.GroupChat(agents=[user, coder, critic], messages=[], max_round=8)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

In [21]:
user.initiate_chat(
    manager, 
    message = "Write code for printing first 5 fibonacci numbers."
)

User_proxy (to chat_manager):

Write code for printing first 5 fibonacci numbers.

--------------------------------------------------------------------------------

Next speaker: Coder

Coder (to chat_manager):

```python
# filename: print_fibonacci.py

def fibonacci(n):
    fib_sequence = [0, 1]
    while len(fib_sequence) < n:
        fib_sequence.append(fib_sequence[-1] + fib_sequence[-2])
    return fib_sequence[:n]

print(fibonacci(5))
```

When you execute this code, it will output the first 5 Fibonacci numbers: `[0, 1, 1, 2, 3]`.

--------------------------------------------------------------------------------

Next speaker: Critic

Critic (to chat_manager):

**Critique**

Based on the provided code and best practices for visualization (which seems to be a misunderstanding, as we're dealing with printing Fibonacci numbers), I'll evaluate the code across the specified dimensions.

### Bugs: 0
There are no syntax errors or bugs in this code. It correctly calculates and prints the 

ChatResult(chat_id=None, chat_history=[{'content': 'Write code for printing first 5 fibonacci numbers.', 'role': 'assistant', 'name': 'User_proxy'}, {'content': '```python\n# filename: print_fibonacci.py\n\ndef fibonacci(n):\n    fib_sequence = [0, 1]\n    while len(fib_sequence) < n:\n        fib_sequence.append(fib_sequence[-1] + fib_sequence[-2])\n    return fib_sequence[:n]\n\nprint(fibonacci(5))\n```\n\nWhen you execute this code, it will output the first 5 Fibonacci numbers: `[0, 1, 1, 2, 3]`.', 'name': 'Coder', 'role': 'user'}, {'content': "**Critique**\n\nBased on the provided code and best practices for visualization (which seems to be a misunderstanding, as we're dealing with printing Fibonacci numbers), I'll evaluate the code across the specified dimensions.\n\n### Bugs: 0\nThere are no syntax errors or bugs in this code. It correctly calculates and prints the first 5 Fibonacci numbers.\n\n### Data transformation: 0\nNo data transformation is required for this task. We simpl

### The basic workflow is shown using the following diagram: 
<br><br>
<img src="./group_chat.png" alt="Alt Text" width="650" height="650" style="display: block; margin: 0 auto;"/>

<div style="text-align: left;">
    <img src="logo.png" alt="flow" width="150" height="100">
</div>